# Edge classification

This notebooks shows how to classify edges of a graph. In many GNN tracking approaches, we start from an initial graph (e.g., built from a point cloud with the strategy described in `009_build_graphs_ml.ipynb`). We then try to falsify all edges that connected hits of two different particles. If edge classification (EC) would be perfect, we could then reconstruct tracks as connected components of the graph.
For our object condensation approach, EC is only an auxiliary step. Edges are only considered for message passing but are not important for the final decision on how tracks look. However, EC is still important to help the model to learn quickly.

For background on pytorch lightning, see `009_build_graphs_ml.ipynb`.

In [1]:
from pytorch_lightning import Trainer
from torch import nn
from pytorch_lightning.core.mixins import HyperparametersMixin
import torch
from functools import partial

from gnn_tracking.models.graph_construction import MLGraphConstructionFromChkpt

from gnn_tracking.metrics.losses import EdgeWeightFocalLoss
from gnn_tracking.training.ec import ECModule

from gnn_tracking.utils.loading import TrackingDataModule

We can either directly load graphs (from disk), or we load point clouds and build edges on the fly using the module from `009_build_graphs_ml.ipynb`.

## From on-disk graphs

### 1. Setting up the data

If you are not working on Princeton's `della`, you can download these example graphs [here](https://cernbox.cern.ch/s/4xYL99cd7zNe0VK). Note that this is simplified data (pt > 1 GeV truth cut) and a single event has been broken up into 32 sectors.

In [2]:
dm = TrackingDataModule(
    train=dict(
        dirs=[
            "/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/graphs_v1/part1_pt1/all"
        ],
        stop=28000,
        # If you run into memory issues, reduce this
        batch_size=10,
    ),
    val=dict(
        dirs=[
            "/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/graphs_v1/part1_pt1/all"
        ],
        start=28000,
    ),
)

### 2. Defining the module

In [3]:
class SillyEC(nn.Module, HyperparametersMixin):
    def __init__(
        self,
        node_in_dim: int,
        edge_in_dim: int,
        hidden_dim: int = 12,
    ):
        super().__init__()
        self.save_hyperparameters()
        self.node_in_dim = node_in_dim
        self.edge_in_dim = edge_in_dim
        self.hidden_dim = hidden_dim

        self.fcnn = nn.Sequential(
            nn.Linear(edge_in_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid(),
        )

    def forward(self, data):
        w = self.fcnn(data.edge_attr).squeeze()
        return {"W": w}

In [5]:
model = SillyEC(node_in_dim=6, edge_in_dim=4, hidden_dim=128)

### 2. Setting up the loss functions and the lightning module

In [6]:
lmodel = ECModule(
    model=model,
    loss_fct=EdgeWeightFocalLoss(alpha=0.3),
    optimizer=partial(torch.optim.Adam, lr=1e-4),
)

[18:26:19] DEBUG: Got obj of type <class '__main__.SillyEC'>, assuming I have to save hyperparameters
[18:26:19] DEBUG: Saving hyperperameters {'class_path': '__main__.SillyEC', 'init_args': {'node_in_dim': 6, 'edge_in_dim': 4, 'hidden_dim': 128}}
[18:26:19] DEBUG: Got obj of type <class 'gnn_tracking.metrics.losses.EdgeWeightFocalLoss'>, assuming I have to save hyperparameters
[18:26:19] DEBUG: Saving hyperperameters {'class_path': 'gnn_tracking.metrics.losses.EdgeWeightFocalLoss', 'init_args': {'pt_thld': 0.0, 'alpha': 0.3, 'gamma': 2.0, 'pos_weight': tensor([1.])}}


### 3. Starting training

In [7]:
trainer = Trainer(max_epochs=1, accelerator="cpu", log_every_n_steps=1)
trainer.fit(model=lmodel, datamodule=dm)

/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3 ...
  rank_zero_warn(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:176: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but 

Sanity Checking: 0it [00:00, ?it/s]

/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 48 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/torch_geometric/data/collate.py:145: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  storage = elem.storage()._new_shared(numel)
/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/pytorch_lightning/utilities/data.py:76: UserWarning: Trying to infer the `

          Validation epoch=0          
┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━┓
┃ Metric           ┃   Value ┃ Error ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━┩
│ max_ba           │ 0.81810 │   nan │
│ max_ba_loc       │ 0.49246 │   nan │
│ max_f1           │ 0.73951 │   nan │
│ max_f1_loc       │ 0.49246 │   nan │
│ max_mcc          │ 0.70146 │   nan │
│ max_mcc_loc      │ 0.49246 │   nan │
│ roc_auc          │ 0.91724 │   nan │
│ roc_auc_0.001FPR │ 0.49987 │   nan │
│ roc_auc_0.01FPR  │ 0.60937 │   nan │
│ total            │ 0.10031 │   nan │
│ tpr_eq_tnr       │ 0.80920 │   nan │
│ tpr_eq_tnr_loc   │ 0.49009 │   nan │
└──────────────────┴─────────┴───────┘

/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/torch_geometric/data/collate.py:145: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  storage = elem.storage()._new_shared(numel)
/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 48 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


...

/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/torch_geometric/data/collate.py:145: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  storage = elem.storage()._new_shared(numel)


Validation: 0it [00:00, ?it/s]

/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/torch_geometric/data/collate.py:145: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  storage = elem.storage()._new_shared(numel)
/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/pytorch_lightning/utilities/data.py:76: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 292. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/pytorch_lightning/utilities/data.py:76: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 264. To avoid any miscalculations, 

          Validation epoch=0          
┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━┓
┃ Metric           ┃   Value ┃ Error ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━┩
│ max_ba           │ 0.85771 │   nan │
│ max_ba_loc       │ 0.39168 │   nan │
│ max_f1           │ 0.78760 │   nan │
│ max_f1_loc       │ 0.44868 │   nan │
│ max_mcc          │ 0.75603 │   nan │
│ max_mcc_loc      │ 0.45369 │   nan │
│ roc_auc          │ 0.92603 │   nan │
│ roc_auc_0.001FPR │ 0.64743 │   nan │
│ roc_auc_0.01FPR  │ 0.75782 │   nan │
│ total            │ 0.03098 │   nan │
│ tpr_eq_tnr       │ 0.84292 │   nan │
│ tpr_eq_tnr_loc   │ 0.32828 │   nan │
└──────────────────┴─────────┴───────┘

`Trainer.fit` stopped: `max_epochs=1` reached.


## With graphs built on-the-fly from point clouds

Step 1: Configure data module to load point clouds (rather than graphs).
Step 2: Add `MLGraphConstructionFromChkpt` as preproc.

In [ ]:
lmodel = ECModule(
    model=model,
    loss_fct=EdgeWeightFocalLoss(alpha=0.3),
    optimizer=partial(torch.optim.Adam, lr=1e-4),
    preproc=MLGraphConstructionFromChkpt(
        ml_class_name="gnn_tracking.models.graph_construction.GraphConstructionFCNN",
        ml_chkpt_path="/path/to/your/checkpoint",
    ),
)

Instead of `MLGraphConstructionFromChkpt` you can also take a look at `MLGraphConstruction` that simply takes a model (that you can instantiate in any way).